In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.firefox.options import Options
############################
from bs4 import BeautifulSoup
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import os

In [2]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options = options)
driver.get('https://coinmarketcap.com/historical/20230825/')
t = driver.find_elements(By.CSS_SELECTOR, '.cmc-table-row a')
driver.quit

C:\Users\ROG\AppData\Local\Temp\ipykernel_18968\1047215634.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True


<bound method WebDriver.quit of <selenium.webdriver.firefox.webdriver.WebDriver (session="c621f2ac-a71a-441f-83b9-315a79e7f710")>>

In [3]:
urls_list = []
for i in t:
    urls_list.append(i.get_attribute('href'))
first20 = urls_list[:60:3]
first20
others = urls_list[60:]
others
MainLinks = first20 + others
#MainLinks , len(MainLinks)

In [4]:
ranks =list(range(1,201))

In [5]:
names = []
for i in MainLinks:
    names.append(i[37:-1])
#names

In [6]:
HistoricalLinks=[]
for i in MainLinks:
    HistoricalLinks.append(i+'historical-data/')
#HistoricalLinks

In [7]:
def get_url(url):
    res=requests.get(url)
    so= BeautifulSoup(res.text, "html.parser")
    return so
with ThreadPoolExecutor(max_workers=201) as pool:
    t=list(pool.map(get_url,MainLinks))
    #print(t,len(t))

In [8]:
symbol=[]
for i in t:
    symbol.append(i.find(class_='sc-16891c57-0 cjeUNx base-text').text)
#(symbol,len(symbol))

In [9]:
##len list == 0
## len must be 200, may need nulls where no github link

github_link=[]
for i in t:
    k=i.find_all(rel='nofollow noopener')
    d=0
    for j in k:
        if  'https://github.com/' in j.get('href'):
            github_link.append(j.get('href'))
            d=1
            break
    if d==0:
        github_link.append(None)

In [10]:
github_link

['https://github.com/bitcoin/bitcoin',
 'https://github.com/ethereum/wiki/wiki/White-Paper',
 None,
 'https://github.com/bnb-chain',
 'https://github.com/ripple/rippled',
 'https://github.com/centrehq/centre-tokens',
 None,
 'https://github.com/dogecoin/dogecoin/blob/master/README.md',
 'https://github.com/solana-labs',
 'https://github.com/tronprotocol/java-tron',
 'https://github.com/paritytech/polkadot',
 None,
 'https://github.com/maticnetwork/whitepaper/',
 'https://github.com/ton-blockchain',
 'https://github.com/litecoin-project/litecoin',
 'https://github.com/shytoshikusama/woofwoofpaper/raw/main/SHIBA_INU_WOOF_WOOF.pdf',
 'https://github.com/WrappedBTC/bitcoin-token-smart-contracts',
 None,
 None,
 'https://github.com/ava-labs/avalanchego',
 'https://github.com/stellar/stellar-core',
 'https://github.com/smartcontractkit/chainlink',
 'https://github.com/paxosglobal/busd-contract',
 'https://github.com/trusttoken/smart-contracts',
 None,
 'https://github.com/monero-project/rese

In [11]:
tags=[]
for i in t:
    k=i.find_all(class_='sc-16891c57-0 sc-b7faf77f-1 ctYAzo')
    m=[]
    for j in k:
        if j.text not in m:
            m.append(j.text)      
    tags.append(m)

#(tags,len(tags))    

In [12]:
df = pd.DataFrame({'Rank':ranks,
                   'Name':names,
                    'Symbol':symbol,
                    'MainLink':MainLinks,
                    'HistoricalLink':HistoricalLinks,
                    'Tags': tags,
                    'Github_link':github_link})
df

,Rank,Name,Symbol,MainLink,HistoricalLink,Tags,Github_link
0,1,bitcoin,BTC,https://coinmarketcap.com/currencies/bitcoin/,https://coinmarketcap.com/currencies/bitcoin/h...,"[Mineable, PoW, SHA-256]",https://github.com/bitcoin/bitcoin
1,2,ethereum,ETH,https://coinmarketcap.com/currencies/ethereum/,https://coinmarketcap.com/currencies/ethereum/...,"[PoS, Smart Contracts, Ethereum Ecosystem]",https://github.com/ethereum/wiki/wiki/White-Paper
2,3,tether,USDT,https://coinmarketcap.com/currencies/tether/,https://coinmarketcap.com/currencies/tether/hi...,"[Payments, Stablecoin, Asset-Backed Stablecoin]",None
3,4,bnb,BNB,https://coinmarketcap.com/currencies/bnb/,https://coinmarketcap.com/currencies/bnb/histo...,"[Marketplace, Centralized Exchange (CEX) Token...",https://github.com/bnb-chain
4,5,xrp,XRP,https://coinmarketcap.com/currencies/xrp/,https://coinmarketcap.com/currencies/xrp/histo...,"[Medium of Exchange, Enterprise Solutions, Arr...",https://github.com/ripple/rippled
...,...,...,...,...,...,...,...
195,196,playdapp,PLA,https://coinmarketcap.com/currencies/playdapp/,https://coinmarketcap.com/currencies/playdapp/...,"[Gaming, Ethereum Ecosystem, Metaverse]",https://github.com/playdapp
196,197,pundix-new,PUNDIX,https://coinmarketcap.com/currencies/pundix-new/,https://coinmarketcap.com/currencies/pundix-ne...,[],None
197,198,vulcan-forged-pyr,PYR,https://coinmarketcap.com/currencies/vulcan-fo...,https://coinmarketcap.com/currencies/vulcan-fo...,"[Collectibles & NFTs, Gaming, Entertainment]",None
198,199,status,SNT,https://coinmarketcap.com/currencies/status/,https://coinmarketcap.com/currencies/status/hi...,"[Media, Content Creation, Privacy]",https://github.com/status-im


In [16]:
tagsdf = df.Tags.explode().unique().tolist()
tagsdf = pd.DataFrame(tagsdf)
tagsdf.rename(columns={0:'Tags'}, inplace=True)
tagsdf

,Tags
0,Mineable
1,PoW
2,SHA-256
3,PoS
4,Smart Contracts
...,...
123,Alameda Research Portfolio
124,Everscale Ecosystem
125,NaN
126,Entertainment


In [19]:
import numpy as np

CoinTagPivot = pd.DataFrame(
    {
        'Name': np.repeat(df['Name'], list(map(len, df["Tags"].values))),
        'Tags': np.concatenate(df["Tags"].values)
    }
)

CoinTagPivot

,Name,Tags
0,bitcoin,Mineable
0,bitcoin,PoW
0,bitcoin,SHA-256
1,ethereum,PoS
1,ethereum,Smart Contracts
...,...,...
198,status,Media
198,status,Content Creation
198,status,Privacy
199,amp,SEC Security Token


In [20]:
tagsdf.to_csv('Tags.csv', index = False)

In [21]:
CoinTagPivot.to_csv('CoinTagPivot.csv', index = False)

In [22]:
df = df.drop(columns = ['Tags'])
df.to_csv('CoinMarketCapNormalised.csv', index = False)

In [ ]:
################ download csv at once


# def download(url):
#     option = Options()options=option
#     option.headless =True
#     driver_his=webdriver.Firefox()
#     driver_his.get(url)
#     time.sleep(1)
#     driver_his.find_element("xpath", '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]').click()
#     time.sleep(2)
#     driver_his.find_element("xpath", '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]').click()
#     time.sleep(2)
#     driver_his.find_element("xpath", '//*[@id="tippy-1"]/div/div[1]/div/div/div[2]/span/button').click()
#     time.sleep(2)
#     driver_his.find_element("xpath", '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]').click()
#     time.sleep(2)
#     driver_his.quit()
# with ThreadPoolExecutor(max_workers=201) as pool:
#     pool.map(download,HistoricalLinks)



In [ ]:
# download csv one by one


for i in HistoricalLinks:
    option = Options()
    option.headless =True
    driver_his=webdriver.Firefox(options=option)
    driver_his.get(i)#
    time.sleep(7)
    driver_his.find_element("xpath", '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]').click()
    time.sleep(7)
    driver_his.find_element("xpath", '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[2]/ul/li[5]').click()
    time.sleep(7)
    driver_his.find_element("xpath", '//*[@id="tippy-1"]/div/div[1]/div/div/div[2]/span/button').click()
    time.sleep(7)
    driver_his.find_element("xpath", '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]').click()
    time.sleep(5)
    driver_his.quit()